In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import root_mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor


In [2]:
df_tr = pd.read_csv('/Users/dmitrybertnikov/Desktop/ds_learn/work_on_datasets/data/raw/train.csv')
df_tr.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [3]:
df = df_tr.drop(columns='id')

In [4]:
df.head(

).T

,0,1,2,3,4
road_type,urban,urban,rural,highway,rural
num_lanes,2,4,4,4,1
curvature,0.06,0.99,0.63,0.07,0.58
speed_limit,35,35,70,35,60
lighting,daylight,daylight,dim,dim,daylight
weather,rainy,clear,clear,rainy,foggy
road_signs_present,False,True,False,True,False
public_road,True,False,True,True,False
time_of_day,afternoon,evening,morning,morning,evening
holiday,False,True,True,False,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   road_type               517754 non-null  object 
 1   num_lanes               517754 non-null  int64  
 2   curvature               517754 non-null  float64
 3   speed_limit             517754 non-null  int64  
 4   lighting                517754 non-null  object 
 5   weather                 517754 non-null  object 
 6   road_signs_present      517754 non-null  bool   
 7   public_road             517754 non-null  bool   
 8   time_of_day             517754 non-null  object 
 9   holiday                 517754 non-null  bool   
 10  school_season           517754 non-null  bool   
 11  num_reported_accidents  517754 non-null  int64  
 12  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(2), int64(3), object(4)
memory usage: 37.5+ MB


In [6]:
df.isna().sum()

road_type                 0
num_lanes                 0
curvature                 0
speed_limit               0
lighting                  0
weather                   0
road_signs_present        0
public_road               0
time_of_day               0
holiday                   0
school_season             0
num_reported_accidents    0
accident_risk             0
dtype: int64

In [8]:
X = df.drop(columns='accident_risk')
y = df['accident_risk']

In [15]:
cat_features = X.select_dtypes(include='object').columns.tolist()
cat_features
cat_features_indices = [i for i, col in enumerate(X.columns) if col in cat_features]
cat_features_indices

[0, 4, 5, 8]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=42)



In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

def train_models(X_train, X_test, y_train, y_test, cat_features=None, cv_folds=5):
    """
    Обучает несколько моделей и выбирает лучшую по R²
    """
    
    # Автоматически определяем категориальные признаки, если не указаны
    if cat_features is None:
        cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
        print(f"📋 Автоопределены категориальные признаки: {cat_features}")
    
    # Получаем индексы категориальных признаков для CatBoost
    cat_features_indices = [i for i, col in enumerate(X_train.columns) if col in cat_features]
    
    print(f"🎯 Категориальные признаки ({len(cat_features_indices)}): {cat_features_indices}")
    
    models = {
        'CatBoost': CatBoostRegressor(
            cat_features=cat_features_indices,
            verbose=False, 
            random_state=42,
            n_estimators=100
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=100, 
            random_state=42, 
            n_jobs=-1
        ),
        'GradientBoosting': GradientBoostingRegressor(
            n_estimators=100, 
            random_state=42
        ),
    }
    
    results = []  # ИСПРАВЛЕНО: был result, теперь results
    best_score = -np.inf
    best_model = None
    best_model_name = ""
    
    print("\n=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
    print("Модель           | CV R²     | Train R²  | Test R²   | Status")
    print("-" * 55)
    
    for name, model in models.items():
        try:
            # Для моделей кроме CatBoost нужно предварительно закодировать категориальные признаки
            if name != 'CatBoost' and cat_features:
                X_train_encoded = X_train.copy()
                X_test_encoded = X_test.copy()
                
                # Кодируем категориальные признаки
                for feature in cat_features:
                    if feature in X_train_encoded.columns:
                        X_train_encoded[feature] = X_train_encoded[feature].astype('category').cat.codes
                        X_test_encoded[feature] = X_test_encoded[feature].astype('category').cat.codes
                
                X_train_to_use = X_train_encoded
                X_test_to_use = X_test_encoded
            else:
                X_train_to_use = X_train
                X_test_to_use = X_test
            
            # Кросс-валидация
            cv_scores = cross_val_score(model, X_train_to_use, y_train, cv=cv_folds, scoring='r2')
            cv_r2 = cv_scores.mean()
            
            # Обучение
            model.fit(X_train_to_use, y_train)
            
            # Предсказания
            y_train_pred = model.predict(X_train_to_use)
            y_test_pred = model.predict(X_test_to_use)
            
            # Метрики
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            
            # Сохраняем результаты - ИСПРАВЛЕНО название ключей
            results.append({
                'Model': name,
                'CV_R2': cv_r2,
                'Train_R2': train_r2,
                'Test_R2': test_r2,  # ИСПРАВЛЕНО: было Test_R2
                'CV_Std': cv_scores.std(),
                'Model_Object': model
            })
            
            # Проверяем на лучшую модель
            if test_r2 > best_score:
                best_score = test_r2
                best_model = model
                best_model_name = name
            
            print(f"{name:15} | {cv_r2:8.4f}  | {train_r2:8.4f}  | {test_r2:8.4f}  | ✅")
            
        except Exception as e:
            print(f"{name:15} | {'-':8}  | {'-':8}  | {'-':8}  | ❌ {str(e)[:20]}...")
            continue
    
    # Создаем DataFrame - ИСПРАВЛЕНО: было result, теперь results
    if results:  # Проверяем, что есть результаты
        results_df = pd.DataFrame(results)
        
        # Сортируем по Test R² - ИСПРАВЛЕНО: было Test_R2, теперь Test_R2
        results_df = results_df.sort_values('Test_R2', ascending=False).reset_index(drop=True)
        
        print("\n" + "="*55)
        print(f"🎯 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
        print(f"📊 Test R²: {best_score:.4f}")
        print("="*55)
        
        return best_model, best_model_name, results_df
    else:
        print("❌ Ни одна модель не была успешно обучена")
        return None, None, None

# Упрощенная версия для быстрого тестирования
def train_models_simple(X_train, X_test, y_train, y_test, cv_folds=5):
    """
    Упрощенная версия без категориальных признаков
    """
    models = {
        'CatBoost': CatBoostRegressor(verbose=False, random_state=42, n_estimators=100),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    }
    
    results = []
    best_score = -np.inf
    best_model = None
    best_model_name = ""
    
    print("=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
    print("Модель           | CV R²     | Train R²  | Test R²   | Status")
    print("-" * 55)
    
    for name, model in models.items():
        try:
            # Кросс-валидация
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='r2')
            cv_r2 = cv_scores.mean()
            
            # Обучение
            model.fit(X_train, y_train)
            
            # Предсказания
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
            
            # Метрики
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            
            results.append({
                'Model': name,
                'CV_R2': cv_r2,
                'Train_R2': train_r2,
                'Test_R2': test_r2,
                'CV_Std': cv_scores.std(),
                'Model_Object': model
            })
            
            if test_r2 > best_score:
                best_score = test_r2
                best_model = model
                best_model_name = name
            
            print(f"{name:15} | {cv_r2:8.4f}  | {train_r2:8.4f}  | {test_r2:8.4f}  | ✅")
            
        except Exception as e:
            print(f"{name:15} | {'-':8}  | {'-':8}  | {'-':8}  | ❌ {str(e)[:20]}...")
            continue
    
    if results:
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values('Test_R2', ascending=False).reset_index(drop=True)
        
        print("\n" + "="*55)
        print(f"🎯 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
        print(f"📊 Test R²: {best_score:.4f}")
        print("="*55)
        
        return best_model, best_model_name, results_df
    else:
        print("❌ Ни одна модель не была успешно обучена")
        return None, None, None

# Проверка данных перед обучением
def check_data(X_train, X_test, y_train, y_test):
    """
    Проверка корректности данных перед обучением
    """
    print("=== ПРОВЕРКА ДАННЫХ ===")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    print(f"\nТипы данных в X_train:")
    print(X_train.dtypes)
    
    print(f"\nПропущенные значения:")
    print(f"X_train: {X_train.isnull().sum().sum()}")
    print(f"X_test: {X_test.isnull().sum().sum()}")
    print(f"y_train: {y_train.isnull().sum()}")
    print(f"y_test: {y_test.isnull().sum()}")
    
    # Проверяем, есть ли категориальные признаки
    cat_features = X_train.select_dtypes(include=['object', 'category']).columns
    if len(cat_features) > 0:
        print(f"\n📋 Найдены категориальные признаки: {list(cat_features)}")
        return list(cat_features)
    else:
        print(f"\n✅ Все признаки числовые")
        return None

# Полный процесс с проверкой
def full_training_pipeline(X_train, X_test, y_train, y_test):
    """
    Полный пайплайн обучения с проверкой данных
    """
    # 1. Проверяем данные
    cat_features = check_data(X_train, X_test, y_train, y_test)
    
    # 2. Обучаем модели
    if cat_features:
        print(f"\n🎯 Обучаем с учетом категориальных признаков...")
        best_model, best_name, results_df = train_models(
            X_train, X_test, y_train, y_test, cat_features=cat_features
        )
    else:
        print(f"\n🎯 Обучаем модели...")
        best_model, best_name, results_df = train_models_simple(
            X_train, X_test, y_train, y_test
        )
    
    return best_model, best_name, results_df

# Запуск
print("Запуск исправленной функции...")
best_model, best_name, results_df = full_training_pipeline(X_train, X_test, y_train, y_test)

if results_df is not None:
    print(f"\n📊 Финальные результаты:")
    print(results_df[['Model', 'CV_R2', 'Train_R2', 'Test_R2']].round(4))
else:
    print("❌ Обучение не удалось")

Запуск исправленной функции...
=== ПРОВЕРКА ДАННЫХ ===
X_train shape: (388315, 12)
X_test shape: (129439, 12)
y_train shape: (388315,)
y_test shape: (129439,)

Типы данных в X_train:
road_type                  object
num_lanes                   int64
curvature                 float64
speed_limit                 int64
lighting                   object
weather                    object
road_signs_present           bool
public_road                  bool
time_of_day                object
holiday                      bool
school_season                bool
num_reported_accidents      int64
dtype: object

Пропущенные значения:
X_train: 0
X_test: 0
y_train: 0
y_test: 0

📋 Найдены категориальные признаки: ['road_type', 'lighting', 'weather', 'time_of_day']

🎯 Обучаем с учетом категориальных признаков...
🎯 Категориальные признаки (4): [0, 4, 5, 8]

=== СРАВНЕНИЕ МОДЕЛЕЙ ===
Модель           | CV R²     | Train R²  | Test R²   | Status
-------------------------------------------------------
CatBo